### Dependências

In [5]:
pip install SQLAlchemy pyodbc 

### Connection string (SQL Server), Imports e configurações necessárias 
> #### <font color ='red'>Verificar Drivers ODBC instalados</font>
- No windows: Ferramentas Administrativas > Fontes de Dados ODBC.
- No Linux:   /etc/odbcinst.ini e  /home/{NomeDeUsuario}/.odbcinst.ini 

> Caso não houver o driver indicado na próxima célula, instale-o ou mude a string de conexão para bater com o driver instalado no sistema operacional 




In [1]:
import dados
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base # Para usar linguagem DDL
import urllib

# A linha abaixo é um modelo de string de conexão, substituir de acordo com dados necessários
# params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};SERVER=IP_DO_SERVIDOR;DATABASE=NOME_DO_BANCO;UID=USUARIO_BANCO;PWD=SENHA_BANCO")

params = urllib.parse.quote_plus(dados.dados) 

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

Session = sessionmaker(bind=engine) # Para fazer uso do modelo ORM , precisamos de uma sessão, aqui iniciada e passada a engine como parâmetro
session = Session()  # criando o objeto de sessão para ser utilizada nas transações

### Criação de várias tabelas via abstração de classes 

In [31]:
Base = declarative_base()

class User(Base): # Classe de usuario para ser abstraída no banco de dados
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    password = Column(String)

    def __repr__(self):  # Define como o dado será mostrado nas consultas
        return f'User:  {self.name}'

In [10]:
type(Base)

sqlalchemy.ext.declarative.api.DeclarativeMeta

In [9]:
Base.metadata.create_all(engine) # comando para criar todas as tabelas baseadas nas classes


### Criação de tabela após já ter criado outras tabelas

In [10]:
#Base = declarative_base()
class Product(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True)
    name = Column(String)

In [11]:
Product.__table__.create(engine) # criação da tabela

### Adição de registro

In [20]:
user = User(name='John Snow', password='knowsNothing') # Passando dados para adicionar um registro 
session.add(user) # comando "add" da sessão para adicionar o registro
session.commit()  # só após session.commit() o registro é adicionado no banco
# session.flush()  -> guarda oeprações DDL e as coloca como pendentes. As alterações não são feitas, nem ficam visíveis em outras transacoes até que a sessao receba uma instrução commit()

### Execução de consultas

In [37]:
# Método filter_by()

query = session.query(User).filter_by(name='John Snow') # Definição da consulta em si
# Métodos que indicam o que queremos fazer com os dados da consulta
print(query.count()) # Contar quantos registros bateram com o filtro
print(query.all())   # Retorna todos os registros que bateram com o filtro
print(query.first()) # Retorna o primeiro registro que bateu com o filtro


1
[User:  John Snow]
User:  John Snow


In [38]:
# Método filter() 

print(session.query(User).filter(User.name=='John Snow').first())
print(session.query(User).filter(User.name.like('%John%')).first()) # Possível procurar nomes similares ao passado (como o operador LIKE do SQL)

User:  John Snow
User:  John Snow
